## Melbourne Housing Prices Prediction


In [389]:
import numpy as np
import pandas as pd
import catboost as cb


## 1. Collection

In [390]:
full_data = pd.read_csv("./data/Melbourne_housing_FULL.csv")
data = full_data.loc[~full_data.loc[:, "Price"].isnull()]
data

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.79960,144.99840,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.80790,144.99340,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.80930,144.99440,Northern Metropolitan,4019.0
5,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra City Council,-37.79690,144.99690,Northern Metropolitan,4019.0
6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra City Council,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Yarraville,13 Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,...,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34853,Yarraville,29A Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,...,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34854,Yarraville,147A Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,...,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34855,Yarraville,12/37 Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,...,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0


## 2. Cleaning

In [391]:
data=data[~data.duplicated()]
#Lat = data.loc[:, 'Lattitude']
#Lon = data.loc[:, 'Longtitude']
#data=data.drop(columns=['Lattitude', 'Longtitude'])
#filled = pd.concat([Lat.fillna(Lat.mean()), Lon.fillna(Lon.mean())],axis=1)
#data=pd.concat([data, filled], axis=1)
columns_to_check=['Distance',
                          'Postcode',
                          'CouncilArea',
                          'Regionname',
                          'Propertycount']
for column in columns_to_check:
    data = data.loc[~data.loc[:, column].isnull()]
for col in data.columns:
    print(col, sum(data.loc[:, col].isnull()))
data

Suburb 0
Address 0
Rooms 0
Type 0
Price 0
Method 0
SellerG 0
Date 0
Distance 0
Postcode 0
Bedroom2 6438
Bathroom 6444
Car 6821
Landsize 9262
BuildingArea 16588
YearBuilt 15160
CouncilArea 0
Lattitude 6251
Longtitude 6251
Regionname 0
Propertycount 0


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.79960,144.99840,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.80790,144.99340,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.80930,144.99440,Northern Metropolitan,4019.0
5,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra City Council,-37.79690,144.99690,Northern Metropolitan,4019.0
6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra City Council,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Yarraville,13 Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,...,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34853,Yarraville,29A Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,...,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34854,Yarraville,147A Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,...,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34855,Yarraville,12/37 Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,...,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0


In [392]:
columns_with_nulls=[]
for column in data.columns:
    if data.loc[:, column].isnull().any():
        columns_with_nulls.append(column)
        data = data.drop(columns=column)
print(columns_with_nulls)



['Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude']


In [393]:
for col in data.columns:
    print(col, sum(data.loc[:, col].isnull()))
data

Suburb 0
Address 0
Rooms 0
Type 0
Price 0
Method 0
SellerG 0
Date 0
Distance 0
Postcode 0
CouncilArea 0
Regionname 0
Propertycount 0


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,CouncilArea,Regionname,Propertycount
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,Yarra City Council,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,Yarra City Council,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,Yarra City Council,Northern Metropolitan,4019.0
5,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,Yarra City Council,Northern Metropolitan,4019.0
6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,Yarra City Council,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Yarraville,13 Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,Maribyrnong City Council,Western Metropolitan,6543.0
34853,Yarraville,29A Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,Maribyrnong City Council,Western Metropolitan,6543.0
34854,Yarraville,147A Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,Maribyrnong City Council,Western Metropolitan,6543.0
34855,Yarraville,12/37 Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,Maribyrnong City Council,Western Metropolitan,6543.0


## 3. Exploratory Analysis

In [394]:
for col in data.columns:
    print(col,":",data.loc[:,col].nunique())
    print(data.loc[:,col].value_counts())
data=data.drop(columns=["Address","Regionname","Postcode"])


Suburb : 344
Reservoir          727
Bentleigh East     493
Richmond           439
Preston            415
Brunswick          387
                  ... 
croydon              1
Tecoma               1
Cranbourne East      1
Yarra Glen           1
Kalkallo             1
Name: Suburb, Length: 344, dtype: int64
Address : 26748
5 Charles St        4
13 Robinson St      3
1/1 Clarendon St    3
5 Margaret St       3
14 Arthur St        3
                   ..
8 View St           1
3/12 Rondell Av     1
140 Essex St        1
1 Buxton St         1
3 Tarrengower St    1
Name: Address, Length: 26748, dtype: int64
Rooms : 12
3     11956
2      6785
4      5918
5      1209
1      1196
6       137
7        20
8        14
10        5
12        2
9         1
16        1
Name: Rooms, dtype: int64
Type : 3
h    18470
u     5908
t     2866
Name: Type, dtype: int64
Price : 2871
1100000.0    235
600000.0     235
650000.0     219
800000.0     217
1300000.0    210
            ... 
1452000.0      1
2555000.0    

In [395]:
from sklearn.model_selection import train_test_split

X=data.drop(columns="Price")
y=data.loc[:,"Price"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [396]:
import category_encoders as ce

ce_te = ce.TargetEncoder(cols=X.columns)
ce_te.fit(x_train,y_train)
x_train=ce_te.transform(x_train)
x_test=ce_te.transform(x_test)

/Users/hussami/opt/anaconda3/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


## 4. Model Building

In [398]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
reg = LinearRegression().fit(x_train, y_train)
reg_pred = reg.predict(x_train)
rmse = sqrt(mean_squared_error(y_train, reg_pred))
print(rmse)
reg.score(x_test, y_test)

403952.4097928927


0.6160219594297918

In [399]:
import xgboost as xgb
xgbr = xgb.XGBRegressor(max_depth=8,learning_rate=0.1,n_estimators=500,n_jobs=10, colsample_bytree=0.4,subsample=0.7,reg_alpha=10,reg_lambda=20)

xgbr.fit(x_train,y_train)

tr = xgbr.predict(x_train) 
y_predict = xgbr.predict(x_test)

print(xgbr.score(x_train,y_train))
print(xgbr.score(x_test,y_test))
print(sqrt(mean_squared_error(y_train,tr)))
print(sqrt(mean_squared_error(y_test,y_predict)))

0.8510389849614721
0.7468968347968166
247480.00046051247
323246.31050306815


## 5. Iterating

In [404]:
cbr = cb.CatBoostRegressor(learning_rate=0.3,n_estimators=100,max_depth=13)


cbr.fit(x_train,y_train,verbose=0)

tr2 = cbr.predict(x_train) 

y_predict = cbr.predict(x_test)

print(cbr.score(x_train,y_train))
print(cbr.score(x_test,y_test))
print(sqrt(mean_squared_error(y_train,tr2)))

print(sqrt(mean_squared_error(y_test,y_predict) ) )  #error

0.8951992132349162
0.7264818748384008
207580.26792971152
336029.8390224511


In [403]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
RfR = RandomForestRegressor(n_estimators=1000,max_depth =20, criterion = "mse",max_features=7,max_leaf_nodes=1000)
RfR.fit(x_train,y_train)

prediction=RfR.predict(x_test)

rmse=(sqrt(mean_squared_error(y_test,prediction)))
print(RfR.score(x_train,y_train))
print(RfR.score(x_test,y_test))
print(rmse)

0.8887542185317541
0.7269233441098183
335758.54666125233


In [423]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(hidden_layer_sizes=100,random_state=42, max_iter=500,activation='relu', solver='lbfgs', alpha=0.1)
regr.fit(x_train, y_train)
preds = regr.predict(x_test)
rmse=(sqrt(mean_squared_error(y_test,preds)))
print(regr.score(x_train,y_train))
print(regr.score(x_test,y_test))
print(rmse)

0.6826635989399041
0.6883311258816864
358700.1055828099


/Users/hussami/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
